In [128]:
# machine learning model for NFL 2007 season
import pickle
import pandas as pd

In [129]:
# load the csv file
df = pd.read_csv('./pbp_csv/play_by_play_2007.csv')
df.head()

C:\Users\bnugn\AppData\Local\Temp\ipykernel_20164\1076959086.py:2: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,207,208,218,219,220,248,249,253,254,255,260,262,263,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./pbp_csv/play_by_play_2007.csv')


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2007_01_ARI_SF,2007091001,SF,ARI,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,2007_01_ARI_SF,2007091001,SF,ARI,REG,1,SF,home,ARI,...,0,1,1.239341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,58,2007_01_ARI_SF,2007091001,SF,ARI,REG,1,SF,home,ARI,...,0,1,-0.476123,NaN,NaN,NaN,NaN,NaN,0.453444,-45.344436
3,80,2007_01_ARI_SF,2007091001,SF,ARI,REG,1,SF,home,ARI,...,0,1,1.319001,0.77434,4.286007,2.0,0.987546,0.469452,0.628691,37.130868
4,115,2007_01_ARI_SF,2007091001,SF,ARI,REG,1,SF,home,ARI,...,0,1,-0.461107,NaN,NaN,NaN,NaN,NaN,0.448520,NaN


In [130]:
# calculate the team's win percentage
# format by_date as 'yyyy-mm-dd'
def win_percentage(df, team, by_date):
    # get the team's games
    team_wins = 0
    team_losses = 0
    # loop through the games
    for index, row in df.iterrows():
        # check if the game is before the date
        if row['game_date'] >= by_date:
            break
        if row['desc'] == 'END GAME':
            if row['home_team'] == team:
                if row['home_score'] > row['away_score']:
                    team_wins += 1
                else:
                    team_losses += 1
            elif row['away_team'] == team:
                if row['away_score'] > row['home_score']:
                    team_wins += 1
                else:
                    team_losses += 1
    # calculate the win percentage
    if (team_wins + team_losses) == 0:
        win_percentage = 0.0
    else:
        win_percentage = team_wins / (team_wins + team_losses)
    return win_percentage

In [131]:
# loop through the df and add each teams win percentage prior to the current game
# format by_date as 'yyyy-mm-dd'
def add_win_percentage(df):
    # create a new column
    df['home_win_percentage'] = 0.0
    df['away_win_percentage'] = 0.0
    # loop through the games and add the win percentage for each teams prior to the game
    last_game_date = ''
    for index, row in df.iterrows():
        if row['game_date'] != last_game_date:
            last_game_date = row['game_date']
            home_win_percentage = win_percentage(df, row['home_team'], row['game_date'])
            away_win_percentage = win_percentage(df, row['away_team'], row['game_date'])
        row['home_win_percentage'] = home_win_percentage
        row['away_win_percentage'] = away_win_percentage
    return df


In [132]:
df = add_win_percentage(df)
df.head()

# df to pkl for times sake
print('dumping to pickle...')
df.to_pickle('./pbp_2007_WL.pkl')
print('done')

loading to pickle...


# CHECKPOINT

In [289]:
# load pkl
df = pd.read_pickle('./pbp_2007_WL.pkl')

target = df['fixed_drive_result']
df.drop(columns=['fixed_drive_result'], inplace=True)


In [290]:
columns_to_be_dropped = [
    'game_id',
    'old_game_id', 
    'desc', 
    'play_id',
    'game_date',
    'ydsnet',
    'no_score_prob',
    'opp_fg_prob',
    'opp_safety_prob',
    'opp_td_prob',
    'fg_prob',
    'safety_prob',
    'td_prob',
    'extra_point_prob',
    'two_point_conversion_prob',
    'ep',
    'epa',
    'total_home_epa',
    'total_away_epa',
    'total_home_rush_epa',
    'total_away_rush_epa',
    'total_home_pass_epa',
    'total_away_pass_epa',
    'air_epa',
    'yac_epa',
    'comp_air_epa',
    'comp_yac_epa',
    'total_home_comp_air_epa',
    'total_away_comp_air_epa',
    'total_home_comp_yac_epa',
    'total_away_comp_yac_epa',
    'total_home_raw_air_epa',
    'total_away_raw_air_epa',
    'total_home_raw_yac_epa',
    'total_away_raw_yac_epa',
    'wp',
    'def_wp',
    'home_wp',
    'away_wp',
    'wpa',
    'vegas_wpa',
    'vegas_home_wpa',
    'home_wp_post',
    'away_wp_post',
    'vegas_wp',
    'vegas_home_wp',
    'total_home_rush_wpa',
    'total_away_rush_wpa',
    'total_home_pass_wpa',
    'total_away_pass_wpa',
    'air_wpa',
    'yac_wpa',
    'comp_air_wpa',
    'comp_yac_wpa',
    'total_home_comp_air_wpa',
    'total_away_comp_air_wpa',
    'total_home_comp_yac_wpa',
    'total_away_comp_yac_wpa',
    'total_home_raw_air_wpa',
    'total_away_raw_air_wpa',
    'total_home_raw_yac_wpa',
    'total_away_raw_yac_wpa',
    'passer_player_id',
    'passer_player_name',
    'receiver_player_id',
    'receiver_player_name',
    'rusher_player_id',
    'rusher_player_name',
    'lateral_receiver_player_id',
    'lateral_receiver_player_name',
    'lateral_rusher_player_id',
    'lateral_rusher_player_name',
    'lateral_sack_player_id',
    'lateral_sack_player_name',
    'interception_player_id',
    'interception_player_name',
    'lateral_interception_player_id',
    'lateral_interception_player_name',
    'punt_returner_player_id',
    'punt_returner_player_name',
    'lateral_punt_returner_player_id',
    'lateral_punt_returner_player_name',
    'kickoff_returner_player_name',
    'kickoff_returner_player_id',
    'lateral_kickoff_returner_player_id',
    'lateral_kickoff_returner_player_name',
    'punter_player_id',
    'punter_player_name',
    'kicker_player_name',
    'kicker_player_id',
    'own_kickoff_recovery_player_id',
    'own_kickoff_recovery_player_name',
    'blocked_player_id',
    'blocked_player_name',
    'tackle_for_loss_1_player_id',
    'tackle_for_loss_1_player_name',
    'tackle_for_loss_2_player_id',
    'tackle_for_loss_2_player_name',
    'qb_hit_1_player_id',
    'qb_hit_1_player_name',
    'qb_hit_2_player_id',
    'qb_hit_2_player_name',
    'forced_fumble_player_1_player_id',
    'forced_fumble_player_1_player_name',
    'forced_fumble_player_2_player_id',
    'forced_fumble_player_2_player_name',
    'solo_tackle_1_player_id',
    'solo_tackle_1_player_name',
    'solo_tackle_2_player_id',
    'solo_tackle_2_player_name',
    'assist_tackle_1_player_id',
    'assist_tackle_1_player_name',
    'assist_tackle_2_player_id',
    'assist_tackle_2_player_name',
    'assist_tackle_3_player_id',
    'assist_tackle_3_player_name',
    'assist_tackle_4_player_id',
    'assist_tackle_4_player_name',
    'tackle_with_assist_1_player_id',
    'tackle_with_assist_1_player_name',
    'tackle_with_assist_2_player_id',
    'tackle_with_assist_2_player_name',
    'pass_defense_1_player_id',
    'pass_defense_1_player_name',
    'pass_defense_2_player_id',
    'pass_defense_2_player_name',
    'fumbled_1_player_id',
    'fumbled_1_player_name',
    'fumbled_2_player_id',
    'fumbled_2_player_name',
    'fumble_recovery_1_player_id',
    'fumble_recovery_1_player_name',
    'fumble_recovery_2_player_id',
    'fumble_recovery_2_player_name',
    'sack_player_id',
    'sack_player_name',
    'half_sack_1_player_id',
    'half_sack_1_player_name',
    'half_sack_2_player_id',
    'half_sack_2_player_name',
    'penalty_player_id',
    'penalty_player_name',
    'drive_quarter_end',
    'drive_time_of_possession',
    'drive_play_count',
    'drive_first_downs',
    'drive_inside20',
    'drive_ended_with_score',
    'drive_game_clock_end',
    'drive_end_yard_line',
    'drive_play_id_started',
    'drive_play_id_ended',
    'away_score',
    'home_score',
    'result',
    'total',
    'total_line',
    'spread_line',
    'passer',
    'passer_jersey_number',
    'rusher',
    'rusher_jersey_number',
    'receiver',
    'receiver_jersey_number',
    'passer_id',
    'rusher_id',
    'receiver_id',
    'name',
    'jersey_number',
    'id',
    'fantasy_player_id',
    'fantasy_player_name',
    'fantasy',
    'fantasy_id',
    'qb_epa',
    'xyac_epa',
    'pass_oe',

    'nfl_api_id',
    'play_clock',
    'play_deleted',
    'st_play_type',
    'end_clock_time',
    'end_yard_line',
    'drive_end_transition',
    'drive_real_start_time',
    'drive_yards_penalized',
    'drive_end_transition',
    'play_type_nfl',
    'stadium',
    'weather',
    'field_goal_result',
    'kick_distance',
    'extra_point_result',
    'two_point_conv_result',
    'timeout',
    'td_team',
    'td_player_id',
    'td_player_name',
    'lateral_receiving_yards',
    'lateral_rushing_yards',
    'forced_fumble_player_1_team',
    'forced_fumble_player_2_team',
    'solo_tackle_1_team',
    'solo_tackle_2_team',
    'assist_tackle_1_team',
    'assist_tackle_2_team',
    'tackle_with_assist',
    'tackle_with_assist_1_team',
    'tackle_with_assist_2_team',
    'fumbled_1_team',
    'fumbled_2_team',
    'fumble_recovery_1_team',
    'fumble_recovery_2_team',
    'fumble_recovery_1_yards',
    'fumble_recovery_2_yards',
    'return_team',
    'replay_or_challenge_result',
    'safety_player_name',
    'safety_player_id',
    'season',
    'series_success',
    'series_result',
    'order_sequence',
    'start_time',
    'drive_start_yard_line',
    'location',
    'timeout_team',
]

columns_to_fill = ['defteam_score', 'score_differential', 'home_wp_post', 'away_wp_post', 'down',
                            'kick_distance', 'passing_yards', 'receiver_player_id', 'receiver_player_name',
                            'receiving_yards', 'rusher_player_id', 'rusher_player_name', 'rushing_yards',
                            'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty',
                            'third_down_converted', 'third_down_failed', 'fourth_down_converted',
                            'fourth_down_failed', 'incomplete_pass', 'touchback', 'interception',
                            'punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed',
                            'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone',
                            'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'fumble_forced',
                            'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 'penalty',
                            'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td',
                            'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown',
                            'rush_touchdown', 'return_touchdown', 'extra_point_attempt', 'two_point_attempt',
                            'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 'complete_pass',
                            'assist_tackle', 'lateral_reception', 'lateral_rush', 'lateral_return', 'lateral_recovery',
                            'defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt',
                            'defensive_extra_point_conv', 'penalty_yards', 'cp', 'cpoe', 'wind', 'xyac_mean_yardage',
                            'xyac_median_yardage', 'xyac_success', 'xyac_fd', 'xpass', 'air_yards', 'yards_after_catch',
                            'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining',
                            ]

columns_to_fill_with_none = ['penalty_team', 'penalty_type', 'pass_length', 'pass_location', 'run_location', 'run_gap',]


In [291]:
# normalize time, time_of_day and drive_game_clock_start
def normalize_time(df):
    for column in ['time', 'time_of_day', 'drive_game_clock_start']:
        df[column] = df[column].apply(lambda x: str(x).split(':')[0])
    return df

df = normalize_time(df)

In [292]:
# function to normalize seconds remaining
def seconds_remaining(sr, total_seconds):
    if sr > total_seconds - 100:
        sr_range = f"{total_seconds - 99}-{total_seconds}"
    else:
        sr_range = f"{sr // 100 * 100 + 1}-{(sr // 100 + 1) * 100}"
    return sr_range

In [293]:
# normalize seconds remaining
# normalize seconds remaining
def normalize_seconds_remaining(df):
    for key, total_seconds in zip(['quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining'], [900, 1800, 3600]):
        df[key] = df[key].apply(lambda x: seconds_remaining(x, total_seconds))

    return df

In [294]:
# drop the columns that are not needed
df[columns_to_fill] = df[columns_to_fill].fillna(0)
df[columns_to_fill_with_none] = df[columns_to_fill_with_none].fillna('None')
df.drop(columns=columns_to_be_dropped, inplace=True)
df = normalize_time(df)
df = normalize_seconds_remaining(df)
df.head()

TypeError: seconds_remaining() missing 1 required positional argument: 'total_seconds'

In [ ]:
# write the df to a csv to check
df.to_csv('./data.csv', index=False)